# Inference User

## Import dependencies

In [1]:
import os
from avato import Client
from avato import Secret
from avato_tflite_dynamic import TFLITEDYNAMIC_Instance

## Login to avato

In [2]:
inference_user_client = Client(
    username=os.environ["INFERENCE_USER_ID"],
    password=os.environ["INFERENCE_USER_PASSWORD"],
    instance_types=[TFLITEDYNAMIC_Instance],
    backend_host="avato-backend.westeurope.cloudapp.azure.com"
)

## Get TFLite instance

In [5]:
# Here the inference user would be given the instance id
# by the model owner.
INSTANCE_ID = "e9fac2a0-aa56-4ecf-80a2-0f59cdb436b6"
inference_user_instance = inference_user_client.get_instance(
    INSTANCE_ID
)

## Check security guarantees

In [6]:
# The inference user performs the same security
# checks that the owner did
inference_user_instance.validate_fatquote(
    expected_measurement="958c8ed1ec4f5d818157acfc3b50a8d9306db94258d3e435abd5e96c72ee081c",
    accept_debug=True,
    accept_group_out_of_date=True,

)

## Creating (randomly) a public-private keypair and setting it

In [13]:
inference_user_secret = Secret()
inference_user_instance.set_secret(inference_user_secret)

## Get the model format

In [14]:
#  Some relevant fields explained in more detail:
# `modelSha256`: Provides a hash of the executed remote model. This adds some notion of identity.
#  `inputTensorFormats`: Format of the input tensors.
#  `outputTensorFormats`: Format of the output tensors.
model_format = inference_user_instance.get_model_format()
print(model_format)

modelSha256: "\375Tr\302*\016\037\241+\207\306.\033\234\341iU5\257\014\353\020h\370\355\001\321\010$\301A\'"
modelSize: 408612
arenaSize: 131072
inputTensorFormats {
  dimensions: 1
  dimensions: 28
  dimensions: 28
  valueType: "float32le"
  size: 3136
}
outputTensorFormats {
  dimensions: 1
  dimensions: 10
  valueType: "float32le"
  size: 40
}



## Make a prediction

In [15]:
from tensorflow import keras

# Import the Fashion MNIST dataset.
fashion_mnist = keras.datasets.fashion_mnist
(_, _), (test_images, _) = fashion_mnist.load_data()
# Preprocess the data: Data normalization.
test_images = test_images / 255.0
test_image = test_images[0:1, :, :]


# Make a prediction using the model running in the secure enclave
prediction_remote = inference_user_instance.predict(test_image)
print(
    f"Remote - Label: {prediction_remote.argmax()} with weight: {prediction_remote.max()}"
)

Remote - Label: 9 with weight: 0.9990992546081543
